##How to calculate anomaly?

In [ ]:
import numpy as np

# Sample data
data = [10, 12, 12, 14, 14, 100, 15, 13, 11, 10]

# Calculate mean and standard deviation
mean = np.mean(data)
std_dev = np.std(data)

# Calculate anomaly
anom = [(x - mean) for x in data]

# Calculate standardized anomaly
anom_std = [(x - mean) / std_dev for x in data]

In [ ]:
mean

In [ ]:
std_dev

In [ ]:
anom

In [ ]:
anom_std

##Calculate the Oceanic Niño Index (ONI) using HadISST dataset

	1.	Obtain the SST Data
The Hadley Center’s HadISST dataset can be downloaded from their official site or accessed via netCDF format (https://www.metoffice.gov.uk/hadobs/hadisst/data/download.html).

	2.	Define the Niño 3.4 Region
  The Niño 3.4 region is bounded by:
  Latitude: 5°N to 5°S
  Longitude: 170°W to 120°W (equivalent to 190°E to 240°E in 0–360° longitude).

	3.	Calculate Monthly SST Anomalies
  For each month, calculate the average SST within the Niño 3.4 region.
	•	Compute a climatology (average SST for each month) based on a 30-year base period (typically 1981–2010 or another standard).
	•	Subtract the monthly climatology from the monthly SSTs to obtain SST anomalies.

	4.	Compute the 3-Month Running Mean
  Apply a 3-month moving average to the SST anomalies. This will smooth out short-term variability, leaving the longer-term trends and anomalies.

	5.	Determine ONI
  The ONI is the 3-month running mean of SST anomalies in the Niño 3.4 region. If the ONI exceeds ±0.5°C for several consecutive months, it indicates the presence of El Niño or La Niña conditions.


In [ ]:
!wget https://www.metoffice.gov.uk/hadobs/hadisst/data/HadISST_sst.nc.gz

--2024-09-09 07:40:44--  https://www.metoffice.gov.uk/hadobs/hadisst/data/HadISST_sst.nc.gz
Resolving www.metoffice.gov.uk (www.metoffice.gov.uk)... 23.220.230.81
Connecting to www.metoffice.gov.uk (www.metoffice.gov.uk)|23.220.230.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239672925 (229M) [application/x-gzip]
Saving to: ‘HadISST_sst.nc.gz’

HadISST_sst.nc.gz   100%[===================>] 228.57M   187MB/s    in 1.2s    

2024-09-09 07:40:46 (187 MB/s) - ‘HadISST_sst.nc.gz’ saved [239672925/239672925]



In [ ]:
!gzip -d HadISST_sst.nc.gz

In [ ]:
import xarray as xr
import pandas as pd

# Load the SST data (e.g., HadISST dataset in netCDF format)
# Assume the SST data is in a file called 'hadisst.nc' with dimensions [time, lat, lon]
ds = xr.open_dataset('HadISST_sst.nc')

# Define the Niño 3.4 region (5N to 5S, 170W to 120W)
nino34_sst = ds['sst'].sel(latitude=slice(5, -5), longitude=slice(-170, -120))


In [ ]:
ds

<xarray.Dataset>
Dimensions:    (time: 1853, nv: 2, latitude: 180, longitude: 360)
Coordinates:
  * time       (time) datetime64[ns] 1870-01-16T11:59:59.505615234 ... 2024-0...
  * latitude   (latitude) float32 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) float32 ...
    sst        (time, latitude, longitude) float32 ...
Attributes:
    Title:                      Monthly version of HadISST sea surface temper...
    description:                HadISST 1.1 monthly average sea surface tempe...
    institution:                Met Office Hadley Centre
    source:                     HadISST
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    Conventions:                CF-1.0
    history:                    13/7/2024 converted to netcdf from pp format
    supplementary_information:  Updates and supplementary information will be...
    comment:                    Data restrictions: for academic research use ...

In [ ]:
nino34_sst

<xarray.DataArray 'sst' (time: 1853, latitude: 10, longitude: 50)>
[926500 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1870-01-16T11:59:59.505615234 ... 2024-0...
  * latitude   (latitude) float32 4.5 3.5 2.5 1.5 0.5 -0.5 -1.5 -2.5 -3.5 -4.5
  * longitude  (longitude) float32 -169.5 -168.5 -167.5 ... -122.5 -121.5 -120.5
Attributes:
    standard_name:  sea_surface_temperature
    long_name:      sst
    units:          C
    cell_methods:   time: lat: lon: mean

In [ ]:

# Calculate the monthly climatology (1981-2010 as an example)
climatology_period = nino34_sst.sel(time=slice('1981-01-01', '2010-12-31'))
monthly_climatology = climatology_period.groupby('time.month').mean('time')